In [1]:
import numpy as np
import pandas as pd
import scipy, numpy, math

## Task 2 

In the problem of choosing one alternative out of three, calculate the Nitzan-Kelly index and the average manipulability index for the Borda count and the number of agents ( k = 3 ).

In [2]:
# set of all possible preferences
patterns = ['ABC', 'ACB', 'BAC', 'BCA', 'CAB', 'CBA']

# code the position of an alternative - through Borda points
# idea: point = {preferences: [points for alternative B, points for alternative A, points for alternative C]}
scores = {}
# those are individual Borda scores
for a in patterns:
    scores[a] = np.zeros(3)

i = 0    
for point in scores.keys():
    scores[point][0] = (point[0] == 'A') * 3 + (point[1] == 'A') * 2 + (point[2] == 'A') 
    scores[point][1] = (point[0] == 'B') * 3 + (point[1] == 'B') * 2 + (point[2] == 'B') 
    scores[point][2] = (point[0] == 'C') * 3 + (point[1] == 'C') * 2 + (point[2] == 'C')  
scores



{'ABC': array([3., 2., 1.]),
 'ACB': array([3., 1., 2.]),
 'BAC': array([2., 3., 1.]),
 'BCA': array([1., 3., 2.]),
 'CAB': array([2., 1., 3.]),
 'CBA': array([1., 2., 3.])}

'BAC': array([2., 3., 1.]) - for preferances 'BAC', A gets 2 poitns, B - 3, C - 1

In [3]:
matrix_deviation = np.full((6, 6, 6), False)
matrix_ans = np.full((6, 6, 6), 'A')




for i in range(6):
    for j in range(6):
        for k in range(6):
            # get total Borda scores
            bord = scores[patterns[i]] + scores[patterns[j]] + scores[patterns[k]]
            max_value = np.max(bord)
            max_indices = np.where(bord == max_value)[0][0]
            # get the matrix of winning alternatives (max Borda points)
            matrix_ans[i][j][k] = chr(max_indices + ord('A'))


# now calculate possible deviations
for i in range(6):
    for j in range(6):
        for k in range(6):
            # first condition - the chosen alternative is different from the 
            # preferred alternative of the player
            if matrix_ans[i][j][k] != patterns[i][0]:
                # search whether individ has an opportunity to do the manipulation
                # to get their desired alternative on the first place
                # (search is done on the set of all posibble winning alternative
                # for all of the player's preferences)
                if patterns[i][0] in matrix_ans[:][j][k]:
                    matrix_deviation[i][j][k] = True
                # now see the possible manipulations with getting the player's secondly desired 
                # alternative on the position above
                elif matrix_ans[i][j][k] != patterns[i][1] and patterns[i][1] in matrix_ans[:][j][k]:
                    matrix_deviation[i][j][k] = True
            # repeat for all players
            if matrix_ans[i][j][k] != patterns[j][0]:
                if patterns[j][0] in matrix_ans[i][:][k]:
                    matrix_deviation[i][j][k] = True
                elif matrix_ans[i][j][k] != patterns[j][1] and patterns[j][1] in matrix_ans[i][:][k]:
                    matrix_deviation[i][j][k] = True

            
            if matrix_ans[i][j][k] != patterns[k][0]:
                if patterns[k][0] in matrix_ans[i][j][:]:
                    matrix_deviation[i][j][k] = True
                elif matrix_ans[i][j][k] != patterns[k][1] and patterns[k][1] in matrix_ans[i][j][:]:
                    matrix_deviation[i][j][k] = True

          

            
matrix_deviation

array([[[False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False,  True, False, False,  True],
        [False, False, False, False,  True, False],
        [False, False, False,  True,  True, False],
        [False, False,  True, False, False,  True]],

       [[False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False,  True, False, False,  True],
        [False, False, False, False,  True, False],
        [False, False, False,  True,  True, False],
        [False, False,  True, False, False, False]],

       [[False, False,  True, False, False,  True],
        [False, False,  True, False, False,  True],
        [ True,  True, False, False,  True, False],
        [False, False, False, False, False, False],
        [False, False,  True, False,  True, False],
        [ True,  True, False, False, False,  True]],

       [[False, False, False, False,  True, False],
      

In [4]:
# now compute indexes
# np.sum(matrix_deviation) = # of deviations
k = 3 # measure of the set of alternatives
n = 3 # number of platers

# Nitzan-Kelly index
IndexNK = np.sum(matrix_deviation) / (math.factorial(k) ** n)
# Average manipulability index
IndexAEM = np.sum(matrix_deviation) / ((math.factorial(k) ** n) * n)

print(f''' Nitzan-Kelly index: {round(IndexNK, 4)},
\n Average manipulability index: {round(IndexAEM, 4)},
\n Number of deviations: {np.sum(matrix_deviation)}''')

 Nitzan-Kelly index: 0.2361,

 Average manipulability index: 0.0787,

 Number of deviations: 51


## Task 3

Consider the problem of choosing one alternative from the set of alternatives  A = {1, 2, 3} by k agents. The election organizer wants the selection rule to satisfy the properties of unanimity and anonymity. As a measure of the manipulability of the mechanism, he considers the number P of preference profiles in which manipulation is possible. Consider several small values of k and find (possibly with the help of a computer program) the minimal value of P = P(k) across all mechanisms. Formulate a hypothesis about some reasonable lower bound for P(k) based on the estimates you have obtained.

In [6]:
k = 4
import itertools
comparisons = {pattern: np.zeros(3) for pattern in patterns}

# here we vectorize pair comparisons (by cases)
for pattern in scores:
    comparisons[pattern][0] = scores[pattern][0] > scores[pattern][1]
    comparisons[pattern][1] = scores[pattern][0] > scores[pattern][2]
    comparisons[pattern][2] = scores[pattern][1] > scores[pattern][2]


matrix_combinations = np.full([len(patterns)] * k, '^^^')
element_combinations = []

# compute all possible combinations
all_combinations = itertools.combinations_with_replacement(range(len(patterns)), k)

total = 1
for combination in all_combinations:
    # here we create a set of possible wiining alternatives and then discard those whicg cannot be chosen
    # according to pair-wise comprisons (by all players)
    test_results = np.zeros(3)
    
    for i in combination:
        test_results += comparisons[patterns[i]]
    
    remaining_elements = {'A', 'B', 'C'}
    
    if test_results[0] == k:
        remaining_elements.discard('B')
    elif test_results[0] == 0:
        remaining_elements.discard('A')
    
    if test_results[1] == k:
        remaining_elements.discard('C')
    elif test_results[1] == 0:
        remaining_elements.discard('A')
    
    if test_results[2] == k:
        remaining_elements.discard('C')
    elif test_results[2] == 0:
        remaining_elements.discard('B')
    
    matrix_combinations[combination] = ''.join(remaining_elements)
    element_combinations.append(''.join(remaining_elements))
    total *= len(''.join(remaining_elements))
matrix_combinations

array([[[['A', 'A', 'BA', 'BA', 'AC', 'BAC'],
         ['^^^', 'A', 'BA', 'BAC', 'AC', 'BAC'],
         ['^^^', '^^^', 'BA', 'BA', 'BAC', 'BAC'],
         ['^^^', '^^^', '^^^', 'BA', 'BAC', 'BAC'],
         ['^^^', '^^^', '^^^', '^^^', 'AC', 'BAC'],
         ['^^^', '^^^', '^^^', '^^^', '^^^', 'BAC']],

        [['^^^', '^^^', '^^^', '^^^', '^^^', '^^^'],
         ['^^^', 'A', 'BA', 'BAC', 'AC', 'BAC'],
         ['^^^', '^^^', 'BA', 'BAC', 'BAC', 'BAC'],
         ['^^^', '^^^', '^^^', 'BAC', 'BAC', 'BAC'],
         ['^^^', '^^^', '^^^', '^^^', 'AC', 'BAC'],
         ['^^^', '^^^', '^^^', '^^^', '^^^', 'BAC']],

        [['^^^', '^^^', '^^^', '^^^', '^^^', '^^^'],
         ['^^^', '^^^', '^^^', '^^^', '^^^', '^^^'],
         ['^^^', '^^^', 'BA', 'BA', 'BAC', 'BAC'],
         ['^^^', '^^^', '^^^', 'BA', 'BAC', 'BAC'],
         ['^^^', '^^^', '^^^', '^^^', 'BAC', 'BAC'],
         ['^^^', '^^^', '^^^', '^^^', '^^^', 'BAC']],

        [['^^^', '^^^', '^^^', '^^^', '^^^', '^^^'],
         ['

This is a reduced profiles matrix for further iterations. The only problem is that it will take the following amount of years to completely iterate:

In [7]:
years = total / (24 * 60 * 60 * 365 * 5 * 10^5)
years

6.895662601911047e+35